# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [23]:
# pre-given imports in project template
from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice

# added imports
import pandas as pd
import re
import requests
import random
import itertools
import math

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [3]:

def dna_to_rna(s):
    conversion = {'A': 'A', 'C': 'C', 'G': 'G', 'T': 'U'}
    return "".join(conversion[char] for char in s)
    
if __name__ == '__main__':
    print(dna_to_rna("AACGTGATTTC"))

AACGUGAUUUC


### Idea of solution

Idea is, like instructed, to use a dictionary to store how each molecule is mapped when transcribed into RNA. Then, the input sequence is iterated and on each iteration the mapped value of the molecule is added to the output string.

### Discussion

The simple function works as expected. There could have been a one-liner for the solution, but using a dictionary it is quite simple to see what the function does.

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [4]:
def get_html():
    url = 'https://raw.githubusercontent.com/csmastersUH/data_analysis_with_python_2020/master/Codon%20usage%20table.html'
    html_content = requests.get(url).text
        
    return html_content
    

def get_data():
    html = ''#get_html()
    with open('proteincodons.html') as file:
        html = file.read()

    regexp = re.compile('<pre>(.*?)</pre>', re.MULTILINE | re.DOTALL)
    protein_section = re.findall(regexp, html)[0]
    
    # cleanup of redundant spaces confusing split
    protein_section = protein_section.replace('( ', '(')
    protein_section_list = protein_section.split()
    
    rows = []
    i = 0
    while i < len(protein_section_list):
        row = [item for item in protein_section_list[i:i+5]]
        # convert strs to numeric where necessary
        row[2] = float(row[2])
        row[3] = float(row[3])
        row[4] = int(row[4].replace('(', '').replace(')', ''))
        rows.append(row)
        i += 5
    data = pd.DataFrame(rows, columns=['triplet', 'amino acid', 'fraction', 'frequency: per thousand', 'number'])
    return data
        

def get_dict():
    codons = {}
    data = get_data()
    
    for i in range(data.shape[0]):
        codons[data['triplet'][i]] = data['amino acid'][i]

    return codons
    
if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}


### Idea of solution

The solution reads the HTML file (file was saved previously into src folder with the name proteincodons.html) into a string. Then, using the re module, a regular expression is used that finds the string between the PRE tags. Dotall is used to include newline as a character matched with '.', and multiline is used to match the regular expression on multiple lines. Then, the string is converted into an array that is then iterated in sets of 5 (length of data for one triplet), and saved into an array of rows that is then used to create a pandas dataframe to store the data. Then, the dataframe is iterated and data is converted into the desired dictionary.

### Discussion

The solution works as intended. The dataframe solution was selected because it feels like a natural way to store the data. This way it is easier to use the data later in the project.

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [5]:

def get_dict_list():
    codons_per_rna = get_dict()
    codons_per_prot = {}
    
    for rna, protein in codons_per_rna.items():
        if not protein in codons_per_prot.keys():
            codons_per_prot[protein] = [rna]
        else:
            codons_per_prot[protein].append(rna)
    return codons_per_prot
    
if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)

{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


### Idea of solution

The solution uses the dict created in the previous exercise. The previous dict is iterated and per each key and value pair it is checked if the protein is already present in the new dictionary, and according to that the rna codon is either added to a new list as value or appended to an existing list.

### Discussion

The function worked as expected, giving lists of RNA sequences of various lengths for each of the amino acids present in the previous dictionary.

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [6]:

def rna_to_prot(s):
    protein_coding = get_dict()
    rna_triplets = [s[i:i+3] for i in range(0, len(s), 3)]
    return "".join(protein_coding[triplet] for triplet in rna_triplets)

def dna_to_prot(s):
    return rna_to_prot(dna_to_rna(s))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))

MISSTM*


### Idea of solution

The solution uses the RNA-to-protein dictionary. First, using string slicing, the input is split into sequences of three characters. Then, these triplets are iterated and per triplet the corresponding amino acid from the conversion dictionary is appended to the output string.

### Discussion

The received amino acid chain is the right mapping given the input data fetched above.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [7]:
def get_probabability_dict():
    data = get_data()
    
    total_counts_per_amino = {}
    for i in range(data.shape[0]):
        amino = data['amino acid'][i]
        count = data['number'][i]
        if data['amino acid'][i] in total_counts_per_amino.keys():
            prev = total_counts_per_amino[amino]
            total_counts_per_amino[amino] = prev + count
        else:
            total_counts_per_amino[amino] = count
    
    probabilities = {}
    for i in range(data.shape[0]):
        total_count_of_amino = total_counts_per_amino[data['amino acid'][i]]
        probability = data['number'][i] / total_count_of_amino
        probabilities[data['triplet'][i]] = probability
        
    return probabilities
        
    
if __name__ == '__main__':
    codon_to_prob = get_probabability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items)//6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i*6:6+i*6]
        ))

AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GCA: 0.228121	GCC: 0.399781	GCG: 0.106176	GCU: 0.265922	GGA: 0.249922	GGC: 0.337109
GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306	GUG: 0.463346	GUU: 0.181770
UAA: 0.297019	UAC: 0.556662	UAG: 0.236738	UAU: 0.443338	UCA: 0.150517	UCC: 0.217960
UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.543843	UGG: 1.000000	UGU: 0.456157
UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134


### Idea of solution

The solution uses the function created previously that reads the HTML file into a pandas dataframe. Then, a helper dictionary is constructed that has the amino acids as keys and as values the total count of samples with the amino acid. This dictionary is used when constructing the probability dictionary that iterates over the dataframe (and with it the RNA triplets) and adds as probability the count of the RNA + amino acid combination divided by total count of occurences of the amino acid.

### Discussion

The solution works quickly and has thanks to dataframe usage quite readable code, as column names are visible in accessing data from the dataframe. A not-so-ideal part is that
the solution reads the html data file again, as probably the most elegant solution would be to read the data only once. However, as the amount of data is small and it is only read twice during this notebook, this duplication does not add significant processing costs and removing the duplication would therefore be unnecessary.

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [8]:
class ProteinToMaxRNA:
    
    def __init__(self):
        pass

    def convert(self, s):
        dict_list = get_dict_list()
        probabilities = get_probabability_dict()
        rna_sequence = ''
        for amino in s:
            rnas_translating = dict_list[amino]
            most_probable = 0.0
            most_probable_rna = ''
            for rna in rnas_translating:
                if probabilities[rna] > most_probable or most_probable_rna == '':
                    most_probable = probabilities[rna]
                    most_probable_rna = rna
            rna_sequence += most_probable_rna
        return rna_sequence


if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))

CUGACCCCCAUCCAGAACAGAGCC


### Idea of solution

The solution uses as helpers the list of rnas translating to an amino acid and the probability per rna + amino acid combination dictionary. The function iterates over characters in the amino acid sequence and for each of them finds the most probable rna sequence translating to the amino acid.

### Discussion

The solution is quite simple to read and works as intended, but it is unsure if this is the most efficient way of finding the most probable rnas, as there are a lot of loops in the function.

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [9]:
def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    randn = random.uniform(0, 1)
    cumulative = 0
    for name, probability in dist.items():
        cumulative += probability
        if randn < cumulative:
            return name
    return next(iter(dist))

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))

T, C, T, A, C, T, C, A, C, A, C, C, T, C, G, T, A, C, C, T, T, T, C, T, G, G, C, G, C


### Idea of solution

Using random.uniform, a random number is generated. Then, the event matching the random number is found by iterating over the events while keeping track of the sum of the iterated probabilities. This way the condition random number < cumulative creates the correct intervals where the event should be chosen.

### Discussion

The solution works as intended and seems to give a pretty accurate distribution of events compared to the corresponding probabilities. The used input, for instance, has clearly more C's and T's, which matches the fact that their probabilities are much higher than the ones of the other events.

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [10]:
class ProteinToRandomRNA(object):
    
    def __init__(self):
        pass

    def convert(self, s):
        rna_sequence = ''
        dict_list = get_dict_list()
        probabilities = get_probabability_dict()

        for amino in s:
            rnas = dict_list[amino]
            probability_distribution = dict(zip(rnas, [probabilities[rna] for rna in rnas]))
            rna_sequence += random_event(probability_distribution)
        return rna_sequence
        
if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))

CUGACACCCAUCCAGAACAGGGCC


### Idea of solution

The solution uses again as helpers the list of rnas translating to an amino acid and the probability per rna + amino acid dictionary. The amino acids in the protein are iterated, and for each of them, the event probability distribution dictionary is created. This is then used to get a random event rna from the random_event helper function

### Discussion

The function works as intended, however, with the randomness it is very hard to see if the functioning is correct. A non-ideal part of this solution performance-wise is the fact that the probability distributions are re-created for every amino acid and this distribution is not saved anyhow to be used in case the amino acid reappears in the protein.

jäätiin tähän! tehty 8/22 tehtävää eli 14 jäljellä. ollaan 46% done!

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [11]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting position of a k-window in the sequence.
    For each starting position the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    for i, _ in enumerate(s):
        a_s = 0
        g_s = 0
        c_s = 0
        t_s = 0
        for j in range(k):
            try:
                if s[i+j] == 'A':
                    a_s += 1
                elif s[i+j] == 'G':
                    g_s += 1
                elif s[i+j] == 'C':
                    c_s += 1
                else:
                    t_s += 1
            except IndexError:
                return
        yield {'A': a_s, 'G': g_s, 'C': c_s, 'T': t_s}
        
    
if __name__ == '__main__':
    s = "ACGTTGCA"
    for d in sliding_window(s, 4):
        print(d)

{'A': 1, 'G': 1, 'C': 1, 'T': 1}
{'A': 0, 'G': 1, 'C': 1, 'T': 2}
{'A': 0, 'G': 2, 'C': 0, 'T': 2}
{'A': 0, 'G': 1, 'C': 1, 'T': 2}
{'A': 1, 'G': 1, 'C': 1, 'T': 1}


### Idea of solution

For each possible starting position, a simple switch-type counter is used to calculate the amount of the nucleotides in the sequence. The try-catch block catches the case where the window 'overflows' from the sequence, meaning that all windows fitting in the input sequence have already been processed.

### Discussion

The solution is, if not the simplest, then close-to simplest solution imaginable to the given problem. The code is readable but not very concise, so it is very much possible that there could be solutions better in both conciseness and performance. At least for the tested input it can be easily seen that the dictionaries given by the function are correct.

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [12]:
# helper function for creating a dict of k-mers existing in sequence
def k_mers(s, k):
    k_mers = {}
    for i in range(len(s)-(k)):
        if s[i:i+k] not in k_mers:
            k_mers[s[i:i+k]] = ''
    return k_mers

def context_list(s, k):
    contexts = k_mers(s, k)
    for i in range(k, len(s)):
        preceding_kmer = s[i-k:i]
        contexts[preceding_kmer] += s[i]
    return contexts
    
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

The solution uses first a helper function that iterates over the sequence and adds each unique k-mer-combination (except the last one as it has no following nucleotide) to a dictionary. Then, the sequence is re-iterated and per each nucleotide, its character is appended to the context of the k-mer preceding it.

### Discussion

With a coarse check using the input sequence, the function seems to be working correctly. The implementation is simple as it divides the problem into finding k-mers and after that finding the contexts. It would also have been possible to generate the context sequences by only iterating once through the input, but the solution would have been more complex to understand, and as the operations done in the two iterations are simple, the performance loss with two iterations is neglible.

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [13]:
def context_probabilities(s, k):
    contexts = context_list(s, k)
    probabilities = {}
    
    for kmer, context in contexts.items():
        nucl_counts = {}
        for char in context:
            if not char in nucl_counts.keys():
                nucl_counts[char] = 1
            else:
                nucl_counts[char] += 1
                
        nucl_probabilities = {}
        for char, count in nucl_counts.items():
            nucl_probabilities[char] = count / len(context)
        probabilities[kmer] = nucl_probabilities
    
    return probabilities
if __name__ == '__main__':
    s = "ATGATATCATCGACGATGTAG"
    print(context_probabilities(s, 2))
    print(context_probabilities(s, 0))

{'AT': {'G': 0.4, 'A': 0.2, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5}, 'GA': {'T': 0.6666666666666666, 'C': 0.3333333333333333}, 'TA': {'T': 0.5, 'G': 0.5}, 'TC': {'A': 0.5, 'G': 0.5}, 'CA': {'T': 1.0}, 'CG': {'A': 1.0}, 'AC': {'G': 1.0}, 'GT': {'A': 1.0}}
{'': {'A': 0.3333333333333333, 'T': 0.2857142857142857, 'G': 0.23809523809523808, 'C': 0.14285714285714285}}


### Idea of solution

For each k-mer, the probability dictionary is derived by the following process: first, the solution creates a helper dictionary that contains the counts of the nucleotides in the context. Then this dictionary is used to create the probability dictionary.

### Discussion

Again, there is very surely another way of doing the same thing in a much more concise code. However, this solution also does its job, despite not perhaps being the most readable as it has a high degree of nestedness.

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [14]:
class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def generate(self, n, seed=None):
        output = ''
        for i in range(self.k):
            output += random_event(self.zeroth)
        
        for i in range(self.k, n):
            preceding_kmer = output[i-self.k:i]
            output += random_event(self.kth[preceding_kmer])
        return output[:n]

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
           'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
           'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
           'GA': {'A': 0.0, 'C': 0.3333333333333333, 'T': 0.6666666666666666, 'G': 0.0},
           'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
           'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
           'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
           'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
           'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}}
    n = 10    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))

KeyError: 'AG'

### Idea of solution

The solution uses first the zeroth attribute of the MarkovChain class to randomly generate the first k nucleotides. Then, for the rest of the output, the attribute kth is used to fetch a random event according to the probabilities associated with the preceding k-mer. The return line slices from the output only the first n characters to the return value. This makes sure the output is of right length even in the cases where n < k. In other cases the output is already of length n.

### Discussion

In this case, as random events are again in use, it is impossible to check from the input if the program is working correctly. One problem with the implementation is that it does not prepare for the cases where a k-mer sequence that is not present is randomly generated  in the self.kth -dictionary. This leads to an uncaught keyerror.

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [15]:
def context_pseudo_probabilities(s, k):
    # first generate all possible kmers
    kmers = itertools.product('ACGT', repeat = k)
    prob_dict = {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
    probabilities = {''.join(k): prob_dict for k in kmers}
    
    contexts = context_list(s, k)
    for kmer, context in contexts.items():
        nucl_counts = {'A': 1, 'C': 1, 'G': 1, 'T': 1}
        for char in context:
            nucl_counts[char] += 1

        updated_probabilities = {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}
        for char, count in nucl_counts.items():
            updated_probabilities[char] = count / (len(context) + 4)
            probabilities[kmer] = updated_probabilities

    return probabilities

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    print(f"zeroth: {zeroth}")
    print("\n".join(f"{k}: {dict(v)}" for k, v in kth.items()))
    
    print("\n", MarkovChain(zeroth, kth, k).generate(20))

zeroth: {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666

### Idea of solution

First, all possible k-mers are added into a list using the Itertools library. Then, pseudo probabilities for every nucleotide containing dicts are added as values to a dictionary with k-mers to probabilities. Using the solution from exercise 11, the probabilities for k-mers existing in the input sequence are computed. The frequencies are initialized to one and context length is increased by four to take into account the pseudo counts, otherwise the counting is the same as before. Finally, pseudo probabilities are replaced with the computed probabilities. 

### Discussion

Again it is quite hard to see by just analyzing the result if the probabilities are correct. Notable is in any case that there are quite substantial differences in probabilities, up to 0.3, but the absolute probaility values never seem to exceed 0.4, so the nucleotide frequencies remain quite even (for example there is no longer context containing only one nucleotide). The solution that first initializes pseudo probabilities and then replaces them is at first sight a little confusing and has some magic numbers such as the +4 to the context lenght, but I am unsure if there exists a more readable solution for the problem.

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [22]:
class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        p = 1.0
        from_zeroth = self.k if len(s) >= self.k else len(s) 
        
        for i in range(from_zeroth):
            p *= self.zeroth[s[i]]
        
        for i in range(self.k, len(s)):
            preceding_kmer = s[i-self.k:i]
            p *= self.kth[preceding_kmer][s[i]]
        return p

    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovProb(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {s} is {mc.probability(s)}")

Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10


### Idea of solution

The function starts with probability 1.0 and per each nucleotide in the input sequence, the probability of it occurring is multiplied into the probability of the sequence. The attribute zeroth is used as nucleotide probability for the first k nucleotides and the attribute kth for the rest.

### Discussion

The solution, like the problem, is rather simple and straightforward. For the input, the outputted probability is really small. This is expected because taking into account that there are 21 nucleotides in the input and for each position there are 4 possible nucleotides, the count of unique sequences (4^21) is so large that the probability for any sequence cannot be anything but tiny.

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [26]:
class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def log_probability(self, s):
        p = 0.0
        from_zeroth = self.k if len(s) >= self.k else len(s) 
        
        for i in range(from_zeroth):
            p += math.log(self.zeroth[s[i]], 2)
        
        for i in range(self.k, len(s)):
            preceding_kmer = s[i-self.k:i]
            p += math.log(self.kth[preceding_kmer][s[i]], 2)
        return p
    
if __name__ == '__main__':
    k = 2
    kth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", k)
    zeroth = context_pseudo_probabilities("ATGATATCATCGACGATGTAG", 0)[""]
    mc = MarkovLog(2, zeroth, kth)
    s="ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {s} is {mc.log_probability(s)}")

Log probability of sequence ATGATATCATCGACGATGTAG is -31.717831515538315


### Idea of solution

fill in

### Discussion

fill in

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [17]:
def better_context_probabilities(s, k):
    return {"": ""}

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    print("\n".join(f"{k}: {v}" for k, v in d.items()))

: 


### Idea of solution

fill in

### Discussion

fill in

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [1]:
class SimpleMarkovChain(object):
    def __init__(self, s, k):
        pass

    def generate(self, n, seed=None):
        return "Q"*n
        
if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))

QQQQQQQQQQ


### Idea of solution

fill in

### Discussion

fill in

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [19]:
def kmer_index(s, k):
    return {}

if __name__ == '__main__':
    k=2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i%10) for i in range(len(s))]))
    print(f"\n{k}-mer index is:")
    d=kmer_index(s, k)
    print(dict(d))

Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
{}


### Idea of solution

fill in

### Discussion

fill in

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [20]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    return {"".join(codon): 0 for codon in product("ACGU", repeat=3)}
    
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    return np.nan

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))    
    pass
    
    # Maybe check that converting back to protein results in the same sequence
    pass
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities("<rna sequence>") # placeholder call
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = {"".join(codon): 0 for codon in product("ACGU", repeat=3)} # placeholder dict
    
    # Create a completely random RNA sequence and get the codon probabilities
    pass
    cp_uniform = codon_probabilities("<random rna sequence>") # placeholder call
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

d(original || predicted) = nan
d(predicted || original) = nan

d(original || uniform) = nan
d(uniform || original) = nan

d(predicted || uniform) = nan
d(uniform || predicted) = nan


### Idea of solution

fill in

### Discussion

fill in

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [21]:
def get_stationary_distributions(transition):
    """
    The function get a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    return np.random.rand(2, 4) - 0.5
    
    
if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("\n".join(
        ", ".join(
            f"{pv:+.3f}"
            for pv in p) 
        for p in get_stationary_distributions(transition)))

+0.162, -0.227, +0.388, +0.151
-0.370, +0.057, +0.198, +0.037


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [22]:
def kl_divergences(initial, transition):
    """
    Calculates the the Kullback-Leibler divergences between empirical distributions
    generated using a markov model seeded with an initial distributin and a transition 
    matrix, and the initial distribution.
    Sequences of length [1, 10, 100, 1000, 10000] are generated.
    """
    return zip([1, 10, 100, 1000, 10000], np.random.rand(5))

if __name__ == "__main__":
    transition=np.array([[0.3, 0, 0.7, 0],
                         [0, 0.4, 0, 0.6],
                         [0.35, 0, 0.65, 0],
                         [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[1]
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results: # iterate on prefix lengths in order (1, 10, 100...)
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))

Transition probabilities are:
[[ 0.3   0.    0.7   0.  ]
 [ 0.    0.4   0.    0.6 ]
 [ 0.35  0.    0.65  0.  ]
 [ 0.    0.2   0.    0.8 ]]
Stationary distributions:
[[ 0.02421173 -0.29026212  0.1389851   0.44623574]
 [ 0.24261234  0.09071543  0.35909807 -0.47662849]]
Using [0.24, 0.09, 0.36, -0.48] as initial distribution

KL divergence of stationary distribution prefix of length     1 is 0.60666836
KL divergence of stationary distribution prefix of length    10 is 0.71775085
KL divergence of stationary distribution prefix of length   100 is 0.38115273
KL divergence of stationary distribution prefix of length  1000 is 0.36116144
KL divergence of stationary distribution prefix of length 10000 is 0.22844669


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [23]:
def main(transition, equilibrium_distribution):
    vals = list(zip(np.random.rand(10), np.random.rand(10, 4) - 0.5))
    return zip(np.random.rand(2, 4) - 0.5, 
               [vals[:5], vals[5:]])


if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")
    stationary_distributions = get_stationary_distributions(transition)
    # Uncomment the below line to check that there actually is only one stationary distribution
    # assert len(stationary_distributions) == 1
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)
    for initial_distribution, results in main(transition, equilibrium_distribution):
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        print("\n".join("{:.11f}   {}".format(di, kl) for di, kl in results))

Transition probabilities are:
[[ 0.3   0.1   0.5   0.1 ]
 [ 0.2   0.3   0.15  0.35]
 [ 0.25  0.15  0.2   0.4 ]
 [ 0.35  0.2   0.4   0.05]]
Equilibrium distribution:
[ 0.05549613 -0.0618828  -0.3045492   0.11913485]

Using [-0.3337754   0.45068768 -0.10300145 -0.20088342] as initial distribution:
kl-divergence   empirical distribution
0.25023050057   [ 0.2243623   0.15184879 -0.36068593  0.0442362 ]
0.85018673369   [-0.27420015 -0.1344076  -0.08342149  0.10517491]
0.69016371310   [-0.11457606  0.27115112  0.16359241 -0.1280264 ]
0.14692236781   [ 0.07706555  0.23608696  0.01647088 -0.08926859]
0.98884289212   [-0.06341222  0.45084425  0.3957673  -0.40553139]

Using [-0.14182502  0.21310283  0.21065106  0.45942999] as initial distribution:
kl-divergence   empirical distribution
0.09700875402   [ 0.35101425 -0.18876535  0.25445247 -0.20776396]
0.51559517427   [ 0.31812104 -0.03010157 -0.2113726  -0.49508434]
0.92023091580   [ 0.40268876 -0.22995087 -0.42421415 -0.1464154 ]
0.32096833450  

### Idea of solution

fill in

### Discussion
fill in